In [127]:
from utils import *
import scanpy as sc
import matplotlib.pyplot as plt

In [128]:
counts_df = pd.read_csv(snakemake.input["counts"], index_col=0)
conditions_df = pd.read_csv(snakemake.input["conditions"], index_col=0)
deseq_df = pd.read_csv(snakemake.input["deseq"], index_col=0)

In [87]:
significant = (deseq_df["padj"] <= 0.01) & (abs(deseq_df["log2FoldChange"]) >= 3.0)

In [88]:
counts_sig_df = counts_df.T.loc[significant].T
deseq_sig_df = deseq_df.loc[significant]

In [133]:
conditions_df = conditions_df.sort_values(by=["metastatic", "mean"])
deseq_sig_df = deseq_sig_df.sort_values(by="log2FoldChange")
counts_sig_df = counts_sig_df.loc[conditions_df.index.values.tolist()][deseq_sig_df.index.values.tolist()]

In [134]:
conditions_df

In [135]:
deseq_sig_df

In [136]:
adata = AnnData(X=counts_sig_df.values, var=deseq_sig_df, obs=conditions_df)

In [137]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)

In [140]:
ax = sc.pl.heatmap(adata, var_names=adata.var_names, groupby="metastatic", num_categories=2, show=False)
plt.savefig(snakemake.output["heatmap_plot"])